### 좋아요수 / 댓글 분산 적용 기반 추천(명곡?)
- Input(title, artist) : contains로 찾기 => 결과 songId 한 개만 반환하도록 한다.
- I am 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 -> 가사분석해서 비슷한 노래 찾기 top5
- I am 찾기 -> 가사분석해서 비슷한 노래 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 top5
- 반환된 songId 들의 정보 출력

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/melon_song.csv')
df.head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201.0,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138267,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859
1,35609035,인생은 뷰티풀,김호중,발라드 국내영화,인생은 뷰티풀: 비타돌체,김호중,김호중,20220907.0,https://cdnimg.melon.co.kr/cm2/album/images/11...,197,17322,하나만 알고 둘은\n잘 모르는 사람입니다\n하루는 비틀거리고\n하루는 뒤뚱거리고\n...,457536944 474137187


In [3]:
# 데이터 처리를 위한 작업
df.date.fillna(0, inplace=True)
df['date'] = df.date.astype(int).astype(str)
df.fillna('', inplace=True)
df['comment_like_total'] = df.comment + df.like
df['songId'] = df.songId.astype(str)

In [5]:
# 한글 불용어 처리
with open('data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

In [6]:
from konlpy.tag import Okt
okt = Okt()

In [7]:
lyrics = []
for lyric in df.lyric:
    lyric = lyric.replace('\n', ' ')
    morphs = okt.morphs(lyric, stem=True)
    tmp = [word for word in morphs if word not in stop_words]
    lyrics.append(' '.join(tmp))
df['morphs'] = lyrics

In [8]:
# 학습할 데이터열 생성
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 3

In [9]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)

In [11]:
indices = pd.Series(df.index, index=df.songId)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [13]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(31).tail(30).index
    return df.songId.iloc[song_indices]

In [71]:
# 키워드 바꿔가면서 찾아보기
keyword_title = 'Ditto'
keyword_artist = 'NewJeans'

songIds = df[df.title.str.contains(keyword_title) & df.artist.str.contains(keyword_artist)]

In [72]:
find_songid = songIds.songId.values[0]
a = get_recommendation(find_songid, cosine_sim).to_frame()
a.head()

,songId
4033,36182838
3102,3837683
5377,36263930
4835,36421590
6101,31037943


In [73]:

# df[df['songId'].isin(a.songId[1:6])].to_dict('records')
df[df['songId'].isin(a.songId[1:6])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]

,songId,title,artist,genre,album,lyricist,composer
3102,3837683,Loving U (러빙유),씨스타,댄스,Summer Special 'Loving U',이단옆차기,이단옆차기
3831,36344975,Say My Name,Say Yes!,댄스,BOYS PLANET - ARTIST BATTLE,손다슬 (MUMW),FLUM3N
4835,36421590,이별클럽 (Feat. 이찬혁),Colde (콜드),R&B/Soul,Love Part 2,Colde (콜드),이찬혁
5377,36263930,기다릴게,PLAVE,록/메탈,ASTERUM,PLAVE,EL CAPITXN
6101,31037943,그런 연애 (Feat. 아인 of 모모랜드),그_냥,인디음악 포크/블루스,그런 연애 (Feat. 아인 of 모모랜드),그_냥,그_냥


In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
sim_scores = pd.Series(cosine_sim_total[indices[find_songid]])
sim_scores.sort_values(ascending=False).head(11).tail(10)

In [75]:
# 찾고 싶은 구간 정하기
numbers = df['comment_like_total']
sorted_numbers = np.sort(numbers)
q1 = np.percentile(sorted_numbers, 25)
q2 = np.percentile(sorted_numbers, 50)
filtered_data = df[(df['comment_like_total'] >= q1) & (df['comment_like_total'] < q2)]
filtered_data = filtered_data[['songId', 'comment_like_total']]
filtered_data.songId.values

array(['35837631', '35866717', '35879546', ..., '3080890', '2981597',
       '34183979'], dtype=object)

In [76]:
# 유사도 top 30 중 원하는 구간에 있는 songId 추출(유사도순)
d = a[a['songId'].isin(filtered_data.songId.values)].head(5)
d

,songId
5377,36263930
4835,36421590
6101,31037943
4776,36282049
477,36425393


In [79]:
# filtered_df = df[df['songId'].isin(a.songId.values[:5])].to_dict('records')
filtered_df = df[df['songId'].isin(a.songId.values[:5])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]
filtered_df

,songId,title,artist,genre,album,lyricist,composer
3102,3837683,Loving U (러빙유),씨스타,댄스,Summer Special 'Loving U',이단옆차기,이단옆차기
4033,36182838,ROAR,더보이즈 (THE BOYZ),댄스,THE BOYZ 8th MINI ALBUM [BE AWAKE],KENZIE,DEEZ
4835,36421590,이별클럽 (Feat. 이찬혁),Colde (콜드),R&B/Soul,Love Part 2,Colde (콜드),이찬혁
5377,36263930,기다릴게,PLAVE,록/메탈,ASTERUM,PLAVE,EL CAPITXN
6101,31037943,그런 연애 (Feat. 아인 of 모모랜드),그_냥,인디음악 포크/블루스,그런 연애 (Feat. 아인 of 모모랜드),그_냥,그_냥
